In [39]:
import pandas as pd
from pandas import DataFrame
import requests
import os
import os.path
import datetime as dt
import json
import numpy as np
from datetime import timedelta, datetime
from dateutil import parser
import math
%matplotlib inline
from matplotlib import pyplot as plt

from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings('ignore')

In [40]:
# Load environment variables 
binance_api_key = os.getenv("BINANCE_API_KEY")
binance_secret_key= os.getenv("BINANCE_SECRET_KEY")


In [41]:
type(binance_api_key)

str

In [42]:
type(binance_secret_key)

str

In [43]:
# Set up client 
from binance.client import Client
binance_client = Client(api_key=binance_api_key, api_secret=binance_secret_key)

# Constants
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750

# Create collecting new data function
def minutes_of_new_data(symbol, kline_size, data, source):
#     if len(data) > 0:  old = datetime.strptime('20 Aug 2020', '%d %b %Y') #<-- Can change start date here
    elif source == "binance": old = datetime.strptime('20 Aug 2020', '%d %b %Y')
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    return old, new

# Create function to retreive tickers and timestamps
def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('20 Aug 2020', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

### Coins Ranked by trade Volume according to (coinmartketcap.com): <font color=red> [ETH, EOS, LTC, LINK]</font>

ALL PRICES ARE IN RELATION TO USDT


In [45]:
# Get Ethereum data
eth_data=get_all_binance('ETHUSDT', '1m')
eth_data=eth_data.loc[:,['close']]
eth_data=eth_data.rename(columns={'close':'ETH Close'})

All caught up..!


In [47]:
eth_data.head()

,ETH Close
timestamp,
2020-08-20 00:00:00,408.65000000
2020-08-20 00:01:00,410.19000000
2020-08-20 00:02:00,410.66000000
2020-08-20 00:03:00,410.57000000
2020-08-20 00:04:00,411.53000000


In [49]:
eth_data.to_csv('eth_usd_min.csv')

In [50]:
# Get chainlink data
link_data=get_all_binance('LINKUSDT', '1m')
link_data=link_data.loc[:,['close']]
link_data=link_data.rename(columns={'close':'LINK Close'})

All caught up..!


In [51]:
link_data.head()

,LINK Close
timestamp,
2020-08-20 00:00:00,16.23130000
2020-08-20 00:01:00,16.26730000
2020-08-20 00:02:00,16.27280000
2020-08-20 00:03:00,16.30980000
2020-08-20 00:04:00,16.31820000


In [52]:
link_data.to_csv('link_usd_min.csv')

In [55]:
# Get LITECOIN data
ltc_data=get_all_binance('LTCUSDT', '1m')
ltc_data=ltc_data.loc[:,['close']]
ltc_data=ltc_data.rename(columns={'close':'LTC Close'})

All caught up..!


In [56]:
ltc_data.head()

,LTC Close
timestamp,
2020-08-20 00:00:00,62.20000000
2020-08-20 00:01:00,62.42000000
2020-08-20 00:02:00,62.41000000
2020-08-20 00:03:00,62.65000000
2020-08-20 00:04:00,62.76000000


In [57]:
ltc_data.to_csv('ltc_usd_min.csv')

In [59]:
# Get EOS data
eos_data=get_all_binance('EOSUSDT', '1m')
eos_data=eos_data.loc[:,['close']]
eos_data=eos_data.rename(columns={'close':'EOS Close'})

All caught up..!


In [60]:
eos_data.head()

,EOS Close
timestamp,
2020-08-20 00:00:00,3.35000000
2020-08-20 00:01:00,3.35910000
2020-08-20 00:02:00,3.35790000
2020-08-20 00:03:00,3.36250000
2020-08-20 00:04:00,3.36600000


In [61]:
eos_data.to_csv('eos_btc_min.csv')